In [1]:
import re
import os
import torch
import pydicom
import numpy as np
import random
import pandas as pd
import torch.nn as nn
from torch import optim
from torchvision import transforms
import torchvision
from torchvision.models import inception_v3
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
TRAIN_PATH= r'/kaggle/input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train'
device= 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
def preprocess(path):
    traindf= pd.read_csv('/kaggle/input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train.csv')
    traindf[['ID','Subtype']]= traindf['ID'].str.rsplit(pat='_',n=1,expand=True)
    traindf= traindf.pivot_table(columns='Subtype',values='Label',index='ID').reset_index()
    traindf.replace([np.inf, -np.inf], np.nan, inplace=True)
    traindf['any']= traindf['any'].apply(lambda x : 0.0 if x==1.0 else 1.0)
    
    return sample(traindf)

In [5]:
def sample(traindf):
    not_any= traindf[traindf['any']==1.0]
    epidural = traindf[traindf['epidural']==1.0]
    intraparenchymal = traindf[traindf['intraparenchymal']==1.0]
    intraventricular = traindf[traindf['intraventricular']==1.0]
    subarachnoid = traindf[traindf['subarachnoid']==1.0]
    subdural = traindf[traindf['subdural']==1.0]
    data= [not_any, epidural, intraparenchymal, intraventricular, subarachnoid, subdural]
    lim= min([i.shape[0] for i in data])
    n= not_any.sample(lim*4)
    df= pd.concat([i.sample(lim) for i in data], axis=0)
    df= pd.concat([df,n],axis=0)
    return df

In [6]:
df = preprocess(r'/kaggle/input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train.csv')

In [7]:
class DicomDataset(Dataset):
    def __init__(self, img_dir, df, transform=None, labels=True):
        self.transform = transform
        self.img_dir = img_dir
        self.df = df
        self.labels=labels
        
    def correct_dcm(self, dcm):
        x = dcm.pixel_array + 1000
        px_mode = 4096
        x[x>=px_mode] = x[x>=px_mode] - px_mode
        dcm.PixelData = x.tobytes()
        dcm.RescaleIntercept = -1000

    def window_image(self, dcm, window_center, window_width):

        if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
            self.correct_dcm(dcm)

        img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
        img_min = window_center - window_width // 2
        img_max = window_center + window_width // 2
        img = np.clip(img, img_min, img_max)

        return img

    def bsb_window(self, dcm):
        brain_img = self.window_image(dcm, 40, 80)
        subdural_img = self.window_image(dcm, 80, 200)
        soft_img = self.window_image(dcm, 40, 380)

        brain_img = (brain_img - 0) / 80
        subdural_img = (subdural_img - (-20)) / 200
        soft_img = (soft_img - (-150)) / 380
        bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1,2,0)

        return bsb_img
    
#     def zero_center(self, image):
#         image = image - PIXEL_MEAN
#         return image
    
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        img_path = os.path.join(self.img_dir, self.df.iloc[idx, 0]+'.dcm')
        data = pydicom.read_file(img_path)
        img = self.bsb_window(data)
        
        if self.transform:       
            augmented = self.transform(image=img)
            img = augmented['image']
        
        if self.labels:
            label = torch.tensor(self.df.iloc[idx, 1:],dtype=torch.float64)#.astype(float).to_numpy()
            return {'image': img, 'labels': label} 
#         img = GET WINDOWED, NORMALIZED and SCALED PIXEL ARRAY HERE
        return {'image': img}

In [8]:
import albumentations as A
from albumentations import Compose, CenterCrop, HorizontalFlip, Normalize, RandomRotate90
from albumentations.pytorch import ToTensorV2

transform_train = Compose([CenterCrop(299,299),
                           Normalize(mean=0.5, std=1.0),
                           A.OneOf([
                                RandomRotate90(p=0.3),
                                HorizontalFlip(p=0.3)
                           ]),
                           ToTensorV2()
])

transform_test= Compose([CenterCrop(299,299),
                         Normalize(mean=0.5, std=1.0),
                         ToTensorV2()
])

In [9]:
from sklearn.model_selection import train_test_split

X,Y= train_test_split(df, test_size=0.2, shuffle=True)

train_dataset= DicomDataset(TRAIN_PATH, X, transform=transform_train, labels=True)
test_dataset= DicomDataset(TRAIN_PATH, Y, transform=transform_test, labels=True)

In [10]:
data_loader_train = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
data_loader_test = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [16]:
class GoogLeNet(nn.Module):
    def __init__(self, aux_logits=True, num_classes=1000):
        super(GoogLeNet, self).__init__()
        assert aux_logits == True or aux_logits == False
        self.aux_logits = aux_logits

        # Write in_channels, etc, all explicit in self.conv1, rest will write to
        # make everything as compact as possible, kernel_size=3 instead of (3,3)
        self.conv1 = conv_block(
            in_channels=3,
            out_channels=64,
            kernel_size=7,
            stride=2,
            padding=3,
        )

        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2 = conv_block(64, 192, kernel_size=3, stride=1, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # In this order: in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1pool
        self.skip_3 = conv_block(192, 480 ,kernel_size=1)
        self.inception3a = Inception_block(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception_block(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.skip_4a = conv_block(480, 512 ,kernel_size=1)
        self.inception4a = Inception_block(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception_block(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception_block(512, 128, 128, 256, 24, 64, 64)
        self.skip_4b = conv_block(512, 832, kernel_size=1)
        self.inception4d = Inception_block(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception_block(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.skip_5 = conv_block(832, 1024, kernel_size=1)
        self.inception5a = Inception_block(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception_block(832, 384, 192, 384, 48, 128, 128)

        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.dropout = nn.Dropout(p=0.4)
        self.fc1 = nn.Linear(16384, num_classes)

        if self.aux_logits:
            self.aux1 = InceptionAux(512, num_classes)
            self.aux2 = InceptionAux(528, num_classes)
        else:
            self.aux1 = self.aux2 = None

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        
        s3= self.skip_3(x)
        x = self.inception3a(x)
        x = self.inception3b(x)
        x += s3
        x = self.maxpool3(x)
        
        s4a = self.skip_4a(x)
        x = self.inception4a(x)

        # Auxiliary Softmax classifier 1
        if self.aux_logits and self.training:
            aux1 = self.aux1(x)

        x = self.inception4b(x)
        x += s4a
        x = self.inception4c(x)
        s4b = self.skip_4b(x)
        x = self.inception4d(x)
        
        
        # Auxiliary Softmax classifier 2
        if self.aux_logits and self.training:
            aux2 = self.aux2(x)

        x = self.inception4e(x)
        x += s4b
        x = self.maxpool4(x)
        s5 = self.skip_5(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        x += s5
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        x = self.fc1(x)

        if self.aux_logits and self.training:
            return aux1, aux2, x
        else:
            return x


class Inception_block(nn.Module):
    def __init__(
        self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1pool
    ):
        super(Inception_block, self).__init__()
        self.branch1 = conv_block(in_channels, out_1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            conv_block(in_channels, red_3x3, kernel_size=1),
            conv_block(red_3x3, out_3x3, kernel_size=(3, 3), padding=1),
        )

        self.branch3 = nn.Sequential(
            conv_block(in_channels, red_5x5, kernel_size=1),
            conv_block(red_5x5, out_5x5, kernel_size=5, padding=2),
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            conv_block(in_channels, out_1x1pool, kernel_size=1),
        )

    def forward(self, x):
        return torch.cat(
            [self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], 1
        )


class InceptionAux(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(InceptionAux, self).__init__()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.7)
        self.pool = nn.AvgPool2d(kernel_size=5, stride=3)
        self.conv = conv_block(in_channels, 128, kernel_size=1)
        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.pool(x)
        x = self.conv(x)
        x = x.reshape(x.shape[0], -1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(conv_block, self).__init__()
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        return self.relu(self.batchnorm(self.conv(x)))

In [17]:
classes = 6

model = GoogLeNet(aux_logits=False, num_classes=classes)
model.to(device)

criterion= nn.BCEWithLogitsLoss()
optimizer= optim.Adam(model.parameters(),lr=2e-5)

In [19]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

n_epochs = 50
history = {'accuracy': [], 'precision': [], 'loss': [], 'f1': []}

for epoch in range(n_epochs):
    print('Epoch {}/{}'.format(epoch, n_epochs - 1))
    print('-' * 10)

    model.train()
    tr_loss = 0
    y_true = []
    y_pred = []

    total_batches = len(data_loader_train)
    tk0 = tqdm(data_loader_train, total=total_batches, desc="Iteration", position=0, leave=True)

    for step, batch in enumerate(tk0):
        inputs = batch["image"]
        labels = batch["labels"]

        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)

        outputs = model(inputs)
        
        loss = criterion(outputs, labels)

        loss.backward()

        tr_loss += loss.item()

        optimizer.step()
        optimizer.zero_grad()

        # Collect true and predicted labels for metrics calculation
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(outputs.cpu().detach().numpy())

        if epoch == 1 and step > 6000:
            epoch_loss = tr_loss / 6000
            print('Training Loss: {:.4f}'.format(epoch_loss))
            break
        
    if epoch and (epoch % 10 == 0):
        torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, f'chkpt_{epoch}.tar')
    # Calculate training loss for the epoch
    epoch_loss = tr_loss / len(data_loader_train)
    print('Training Loss: {:.4f}'.format(epoch_loss))
    history['loss'].append(epoch_loss)

    # Calculate and append accuracy, precision, and F1 score to history
#     y_true = np.round(y_true).astype(int)
    y_true= np.argmax(y_true, axis=1)
#     y_pred = np.round(y_pred).astype(int)
    y_pred= np.argmax(y_pred,axis=1)

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    print('Accuracy: {:.4f}'.format(accuracy))
    print('Precision: {:.4f}'.format(precision))
    print('F1 Score: {:.4f}'.format(f1))

    history['accuracy'].append(accuracy)
    history['precision'].append(precision)
    history['f1'].append(f1)

Epoch 0/49
----------


Iteration: 100%|██████████| 787/787 [03:19<00:00,  3.94it/s]


Training Loss: 0.4210
Accuracy: 0.4986
Precision: 0.4234
F1 Score: 0.4401
Epoch 1/49
----------


Iteration: 100%|██████████| 787/787 [02:40<00:00,  4.90it/s]


Training Loss: 0.3677
Accuracy: 0.5661
Precision: 0.5123
F1 Score: 0.5149
Epoch 2/49
----------


...

Training Loss: 0.0872
Accuracy: 0.8256
Precision: 0.8502
F1 Score: 0.8271
Epoch 48/49
----------


Iteration: 100%|██████████| 787/787 [02:42<00:00,  4.84it/s]


Training Loss: 0.0851
Accuracy: 0.8277
Precision: 0.8532
F1 Score: 0.8291
Epoch 49/49
----------


Iteration: 100%|██████████| 787/787 [02:41<00:00,  4.88it/s]

Training Loss: 0.0857
Accuracy: 0.8254
Precision: 0.8502
F1 Score: 0.8267


In [20]:
torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, 'chkpt_final.tar')

In [21]:
dd= pd.DataFrame(history)
dd.to_csv('stats.csv', index=False)